In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

In [ ]:
# 앞에서와 같이 필요한 라이브러리를 불러오고 데이터셋을 로딩한다.
# 캘리포니아 주택 데이터셋을 사용한다.
# 집에 대한 정보들이 주어졌을 때 평균 집값을 나타내는 Target 컬럼을 맞춰야 한다.
california = fetch_california_housing()

df = pd.DataFrame(california.data, columns = california.feature_names)
df["Target"] = california.target
df.tail()

In [ ]:
'''
약 20,600 개의 샘플로 이뤄진 데이터셋이며, 다음 그림과 같이 Target 컬럼을 포함하여 9개의 컬럼으로 이뤄져 있다.
SGD의 개념을 배우기 전, 미니배치를 구성할 때 전체 데이터셋의 샘플을 한꺼번에 집어넣어 학습을 진행했다.
즉, 이전 방식으로 배치사이즈가 20,640인 상태에서 SGD를 수행하게 된다.
하지만, 지금과 같은 데이터셋의 크기라면 이전 방식대로 진행했을 때 메모리가 모자랄 가능성이 높다.
'''
# 데이터의 분포를 파악하기 위해서, 1000개만 임의 추출하여 페어 플롯을 그린다.
sns.pairplot(df.sample(1000))
plt.show()

In [ ]:
'''
봉우리가 여러 개인 멀티모달(multimodal) 분포들도 일부 보이지만, 일괄적으로표준 스케일링(standard scaling)을 적용하도록 한다.
그림에는 잘 보이지 않겠지만, 각 컬럼의 데이터들은 평균이 0이고, 표준편차가 1인 분포의 형태로 바뀌었을 것이다.
다음 코드에서 정답 컬럼을 제외하고 스케일링을 적용하는 것을 확인한다.
'''
scaler = StandardScaler()
scaler.fit(df.values[:,:-1])
df.values[:, :-1] = scaler.transform(df.values[:, :-1])

sns.pairplot(df.sample(1000))
plt.show()

In [ ]:
# 학습 코드 구현
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# 앞에서 정제된 데이터를 파이토치 텐서로 변환하고, 그 크기를 확인한다.
data = torch.from_numpy(df.values).float()

print(data.shape)

In [ ]:
# 입력 데이터와 출력 데이터를 분리하여 x와 y에 저장한다.
x = data[:, :-1] # 모든 행을 가져오되, 마지막 열을 제외한 모든 열을 가져온다.
y = data[:, -1:] # 모든 행을 가져오되, 마지막 열만 가져온다.

print(x.shape, y.shape)

In [ ]:
# 학습에 필요한 세팅 값을 지정한다.
# 모델은 전체 데이터셋의 모든 샘플을 최대 4천번 학습한다.
# 배치사이즈는 256으로 지정하고 학습률은 0.01로 한다.
n_epochs = 4000
batch_size = 256
print_interval = 200
learning_rate = 1e-2

In [ ]:
'''
nn.Sequential 클래스를 활용하여, 심층신경망을 구성한다.
nn.Sequential을 선언할 때, 선형 계층 nn.Linear와 활성 함수 nn.LeakyReLU를 선언하여 넣어준다.
주의할 점은 첫 번째 선형 계층과 마지막 선형 계층은 실제 데이터셋 텐서 x의 크기(8)와, y의 크기(1)를 입출력 크기로 갖도록 정해준다.
또한, 내부의 선형 계층들은 서로 입출력 크기가 호환되도록 되어 있다는 점도 주목해야 한다.
'''

model = nn.Sequential(
    nn.Linear(x.size(-1), 6),
    nn.LeakyReLU(),
    nn.Linear(6, 5),
    nn.LeakyReLU(),
    nn.Linear(5, 4),
    nn.LeakyReLU(),
    nn.Linear(4, 3),
    nn.LeakyReLU(),
    nn.Linear(3, y.size(-1)),
)

print(model)

In [ ]:
# 앞에서 생성한 모델 객체의 파라미터를 학습시킬 옵티마이저를 생성한다.
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# |x| = (total_size, input_dim)
# |y| = (total_size, output_dim)

for i in range(n_epochs):
    # Shuffle the index to feed-forward.
    indices = torch.randperm(x.size(0))
    x_ = torch.index_select(x, dim=0, index=indices)
    y_ = torch.index_select(y, dim=0, index=indices)
    
    x_ = x_.split(batch_size, dim=0)
    y_ = y_.split(batch_size, dim=0)
    # |x_[i]| = (batch_size, input_dim)
    # |y_[i]| = (batch_size, output_dim)
    
    y_hat = []
    total_loss = 0
    
    for x_i, y_i in zip(x_, y_):
        # |x_i| = |x_[i]|
        # |y_i| = |y_[i]|
        y_hat_i = model(x_i)
        loss = F.mse_loss(y_hat_i, y_i)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        
        total_loss += float(loss) # This is very important to prevent memory leak.
        y_hat += [y_hat_i]

    total_loss = total_loss / len(x_)
    if (i + 1) % print_interval == 0:
        print('Epoch %d: loss=%.4e' % (i + 1, total_loss))
    
y_hat = torch.cat(y_hat, dim=0)
y = torch.cat(y_, dim=0)
# |y_hat| = (total_size, output_dim)
# |y| = (total_size, output_dim)

In [ ]:
'''
앞의 코드에서 loss 변수에 담긴 손실 값 텐서를 float 타입캐스팅을 통해 단순 float 타입으로 변환하여, train_loss 변수에 더하는 것을 볼 수 있다.
이 부분도 매우 중요하기에, 코드 상에 주석을 기입했다.

타입캐스팅 이전의 loss 변수는 파이토치 텐서 타입으로 그래디언트를 가지고 있고,
파이토치의 AutoGrad 작동 원리에 의해서 loss 변수가 계산될 때까지 활용된 파이토치 텐서 변수들이 loss 변수에 줄줄이 엮여 있다.
따라서 float 타입캐스팅이 없다면, total_loss도 파이토치 텐서가 될 것이고, total_loss 변수는 해당 에포크의 모든 loss 변수를 엮고 있을 것이다.
결과적으로, total_loss가 메모리에서 없어지지 않는다면, loss 변수와 그에 엮인 텐서 변수들 모두 아직 참조 중인 상태이므로,
파이썬의 가비지컬렉터에 의해 메모리에서 해제되지 않는다. 즉, 메모리 누수가 발생하게 된다.

더욱이, 추후에 진행될 실습처럼 손실 곡선을 그려보기 위해서 total_loss 변수를 따로 저장하기라도 한다면, 학습이 끝날 때까지 학습에 사용된 대부분의
파이토치 텐서 변수가 메모리에서 해제되지 않는 최악의 상황이 발생할 수 있다.
그러므로, 앞과 같은 상황에서는 float 타입캐스팅 또는 detach 함수를 통해, AutoGrad를 하기 위해 연결된 그래프를 잘라내는 작업이 필요하다.
'''

In [ ]:
# 결과 확인
# 결과를 페어 플롯을 통해 확인해보면, 조금 넓게 퍼져있긴 하지만 대체로 중앙을 통과하는 대각선 주변에 점들이 분포하고 있는 것을 확인할 수 있다.
df = pd.DataFrame(torch.cat([y, y_hat], dim=1).detach().numpy(), columns = ["y", "y_hat"])

sns.pairplot(df, height=5)
plt.show()